# Importing the necessary libraries to implement BioFaceNet

In [2]:
#%run ./importLibraries.ipynb
import scipy.io
import pandas as pd
import numpy as np
import torch
from sklearn.decomposition import PCA
import random
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#print(device)

# Hyperparameters

In [3]:
learning_rate = 1e-5
batch_size = 64
epochs = 50
nimages = 50765
batchSizes = 64
blossweight = 1e-4
appweight = 1e-3
#Shadingweight = 1e-5
sparseweight = 1e-5

# Loading Data

In [6]:
illF_mat = scipy.io.loadmat('./util/illF.mat')
illF = torch.tensor(illF_mat['illF'])

illumA_mat = scipy.io.loadmat('./util/illumA.mat')
illumA = torch.tensor(illumA_mat['illumA'])

illumDmeasured_mat = scipy.io.loadmat('./util/illumDmeasured.mat')
illumDmeasured = torch.tensor(illumDmeasured_mat['illumDmeasured'])

newskincolour_mat = scipy.io.loadmat('./util/Newskincolour.mat')
newskincolour = torch.tensor(newskincolour_mat['Newskincolour']).to(device)

Tmatrix_mat = scipy.io.loadmat('./util/Tmatrix.mat')
Tmatrix = torch.tensor(Tmatrix_mat['Tmatrix']).to(device)

XYZspace_mat = scipy.io.loadmat('./util/XYZspace.mat')
XYZspace = torch.tensor(XYZspace_mat['XYZspace'])

rgbCMF_mat = scipy.io.loadmat('./util/rgbCMF.mat')
rgbCMF = (rgbCMF_mat['rgbCMF'])

In [28]:
def CameraSensitivtyPCA(cmf):
    Y = np.zeros((99,28))
    redS = cmf[0,0]
    greenS = cmf[0,1]
    blueS = cmf[0,2]
    Y[:33,:] = redS/np.sum(redS, axis = 1, keepdims = True)
    Y[33:66,:] = greenS/np.sum(greenS, axis = 1, keepdims = True)
    Y[66:,:] = blueS/np.sum(greenS, axis = 1, keepdims = True)
    pca = PCA(n_components = 2)
    principalComponents = pca.fit(Y.T)
    pc = principalComponents.components_
    mu = principalComponents.mean_
    v = principalComponents.singular_values_[:2]
    return pc,mu,v

PCA model for Camera Sensitivities

In [29]:
pc,mu,EV = CameraSensitivtyPCA(rgbCMF)
pc = torch.tensor(pc).to(device)
mu = torch.tensor(mu).to(device)
EV = torch.tensor(EV).to(device)
LightVectorSize = 15
wavelength = 33
bSize = 2

Normalize Illuminations

In [6]:
illF = torch.reshape(illF,(1, 1, 33, 12))

illumDmeasured = torch.transpose(illumDmeasured,0,1)
illumDmeasured = illumDmeasured.reshape(1,1,33,22)

illumA = illumA/torch.sum(illumA)

illumDNorm = torch.zeros(1,1,33,22,dtype = float)
for i in range(22):
    illumDNorm[0,0,:,i] = illumDmeasured[0,0,:,i]/torch.sum(illumDmeasured[0,0,:,i])

illumDNorm = illumDNorm.to(device)

illumFNorm = torch.zeros(1,1,33,12,dtype = float)
for i in range(12):
    illumFNorm[0,0,:,i] = illF[0,0,:,i]/torch.sum(illF[0,0,:,i])

illumFNorm = illumFNorm.to(device)

muim = torch.tensor([129.1863, 104.7624, 93.5940])
muim = muim.reshape(1,1,3)
lossWeights = torch.tensor([1e-3,1e-4,1e-5,1e-5])